In [3]:
import numpy as np
import sklearn as sk
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
from sqlite_api import *

In [6]:
pdb = 'players.db'

player_lbls = ['Player', 'Position', 'Team', 'Season', 'Plays', 'Fantasy', 'Games', 'FPG', 'NumRuns', 
              'RushYrds', 'RushTDs', 'PassAttempts', 'Complete', 'PassYrds', 'PassTDs', 'Fumbles', 'Interceptions']

In [19]:
## Scoring metrics

qb = {'PassYrds': 0.04, # QBs get 0.04 points per passing yard 
        'PassTDs': 4, 
        'Interceptions': -2,
        'RushYrds': 0.1,
        'RushTDs': 6,
        'Fumbles': -2}

wr = {'PassYrds': 0.01,
         'PassTDs': 6,
         'Interceptions': -2,
         'RushYrds': 0.1,
         'RushTDs': 6,
         'Fumbles': -2}

rb = wr.copy()

te = wr.copy()

scores = {'QB': qb,
             'RB': rb,
             'WR': wr,
             'TE': te}

# need 2 points for 2point conversions

In [26]:
def score_row(row, score_metrics):
    score = 0
    for var in score_metrics[row['Position']]:
        print(var)

In [27]:
select_to_df(pdb, 'PlayerSeason', player_lbls, where='WHERE Season=2012 AND Position = \'WR\'') \
    .apply(score_row, args=(scores), axis=1)

TypeError: ('score_row() takes 2 positional arguments but 5 were given', 'occurred at index 0')